In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [29]:
# 데이터셋 및 데이터 로더 설정
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
train_dataset = datasets.ImageFolder(root='E:/PycharmProjects/peslab/GDSC SC/recycle_img/Training', transform=transform)
test_dataset = datasets.ImageFolder(root='E:/PycharmProjects/peslab/GDSC SC/recycle_img/Validation', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [30]:
class VGG16(nn.Module):
    def __init__(self, num_classes=4):
        super(VGG16, self).__init__()
        self.features = models.vgg16(pretrained=True).features
        # 실험적으로 결정한 적절한 입력 차원으로 수정
        self.classifier = nn.Sequential(
            nn.Linear(2048, 4096),  # X와 Y는 마지막 특징 맵의 크기
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [31]:
# 모델 인스턴스 생성
model = VGG16(num_classes=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [32]:
# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## 과적합 방지를 위한 earlystopping 코드 추가

In [34]:
num_epochs = 5
patience = 2
best_accuracy = 0.0
no_improve_epoch = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 검증 데이터셋을 사용하여 모델 평가
    validation_accuracy = evaluate_model(test_loader, model)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {validation_accuracy:.2f}%')

    # 얼리 스톱 체크
    if validation_accuracy > best_accuracy:
        best_accuracy = validation_accuracy
        no_improve_epoch = 0
        torch.save(model.state_dict(), 'best_model.pth')  # 최적 모델 저장
    else:
        no_improve_epoch += 1

    if no_improve_epoch >= patience:
        print("Early stopping")
        break

print("Training complewkdte!")


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

KeyboardInterrupt: 

In [ ]:
num_epochs = 5 # 학습 에폭 수 설정
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    s = 0
    n = 1
    for inputs, labels in train_loader:
        # inputs = inputs.unsqueeze(0).expand(3, -1, -1, -1, -1)
        inputs, labels = inputs.to(device), labels.to(device)
        #print(inputs.shape)
        # inputs = inputs.permute(0, 2, 1, 3, 4)
        optimizer.zero_grad()
        #print(labels)
        # print(labels.shape)
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        # loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        s += len(train_loader)
        print("Completed : ", n, "The rest :", len(train_loader)-n)
        n += 1
print("Training complete!")

In [33]:
# 모델 평가
def evaluate_model(loader, model):
    model.eval()  # 모델을 평가 모드로 전환
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            print(labels)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100
    return accuracy


In [ ]:
# 테스트 데이터로 모델 평가
test_accuracy = evaluate_model(test_loader, model)
print(f"Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
save_path ='E:/PycharmProjects/peslab/GDSC SC/vgg16_recycle_epoch5_weights.pth'
torch.save(model.state_dict(), save_path)
print("Model has been saved.")